In [ ]:
import sys
import os

# Add the path to the project directory
utils_path = os.path.abspath('..')
if utils_path not in sys.path:
    sys.path.append(utils_path)
# Add the path to the directory containing utils to sys.path
utils_path = os.path.abspath('../utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)
print(sys.path)

### Authenticate the client

Instantiate a training and prediction client with your endpoint and keys. 

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
print(os.getenv("TRAINING_KEY"), os.getenv("TRAINING_ENDPOINT"))

# Authentication
credentials = ApiKeyCredentials(in_headers={"Training-key": os.getenv("TRAINING_KEY")})
trainer = CustomVisionTrainingClient(endpoint=os.getenv("TRAINING_ENDPOINT"), credentials=credentials)

# Authentication for prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": os.getenv("PREDICTION_KEY")})
predictor = CustomVisionPredictionClient(endpoint=os.getenv("PREDICTION_ENDPOINT"), credentials=prediction_credentials)


90dad624b6664556accbcfd69e2e170d https://crackdetection.cognitiveservices.azure.com/


### Creat or get the project

In [3]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Project name setup
project_name = "WRB-Bad-Detection"

# Find project by name
print(f"Searching for project '{project_name}'...")
projects = trainer.get_projects()

project_id = None
for project in projects:
    if project.name == project_name:
        project_id = project.id
        break

if project_id:
    print(f"Project '{project_name}' found with ID: {project_id}")
else:
    print(f"No project found with the name '{project_name}'")
    # Create a new project
    print ("Creating project...")
    project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)


Searching for project 'WRB-Bad-Detection'...
Project 'WRB-Bad-Detection' found with ID: 1ee0bb48-3b3f-419d-a575-e12c98f91578


In [4]:
# Get iterations
print(f"Fetching iterations for project '{project_name}'(id:'{project_id}')...")
iterations = trainer.get_iterations(project_id)

# List iteration IDs
print("Iteration:")
for iteration in iterations:
    # print(iteration)
    print(f"{iteration.name}, Created at {iteration.created}, Last modified at {iteration.last_modified}")

Fetching iterations for project 'WRB-Bad-Detection'(id:'1ee0bb48-3b3f-419d-a575-e12c98f91578')...
Iteration:
Iteration 3, Created at 2024-06-29 07:41:11.493000+00:00, Last modified at 2024-06-29 09:13:19.213000+00:00
Iteration 2, Created at 2024-06-29 07:19:07.863000+00:00, Last modified at 2024-06-29 07:48:31.821000+00:00
Iteration 1, Created at 2024-06-28 20:03:03.686000+00:00, Last modified at 2024-06-29 07:27:01.080000+00:00


### Test the prediction endpoint
To send an image to the prediction endpoint and retrieve the prediction, add the following code to the end of the file:

In [ ]:
publish_iteration_name = "detectModel"
# Now there is a trained endpoint that can be used to make a prediction

# Open the sample image and get back the prediction results.
with open("test_image.jpg", mode="rb") as test_image:
    results = predictor.detect_image(
        project.id, 
        publish_iteration_name, 
        test_image)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": \
          {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}"
          .format(prediction.probability * 100, 
                  prediction.bounding_box.left, 
                  prediction.bounding_box.top, 
                  prediction.bounding_box.width, 
                  prediction.bounding_box.height))


### Test dataset

In [ ]:
import pandas as pd

# Read the dataset
# data = pd.read_excel('WRB_trimmed.xlsx')
data = pd.read_excel('../dataset/WRB_All.xlsx')

# test sets based on the 'Group Name' column
test_data = data[data['Group Name'] == 'Test']

### Model Evaluation

Evaluate the model performance on the test data.

In [ ]:
from azure.cognitiveservices.vision.customvision.training.models import CustomVisionErrorException
from sklearn.metrics import precision_score, recall_score, f1_score
from utils.network_functions import fetch_image
import numpy as np

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

def get_bounding_boxes(points):
    x_coords = [p['x'] for p in points]
    y_coords = [p['y'] for p in points]
    min_x = min(x_coords)
    min_y = min(y_coords)
    max_x = max(x_coords)
    max_y = max(y_coords)
    return [min_x, min_y, max_x - min_x, max_y - min_y]

def evaluate_model(test_data, predictor, project_id, iteration_id):
    y_true = []
    y_pred = []

    for _, row in test_data.iterrows():
        img = fetch_image(row['2D Image URL'])
        img.save("test_image.jpg")

        with open("test_image.jpg", "rb") as image_contents:
            results = predictor.detect_image(
                project.id, 
                publish_iteration_name, 
                image_contents.read()
                )

        ground_truth_boxes = [get_bounding_boxes(eval(row['2D Image Points']))]
        predicted_boxes = [[pred.bounding_box.left, pred.bounding_box.top, 
                            pred.bounding_box.width, pred.bounding_box.height] for pred in results.predictions]

        for gt_box in ground_truth_boxes:
            max_iou = 0
            best_pred_box = None
            for pred_box in predicted_boxes:
                iou = calculate_iou(gt_box, pred_box)
                if iou > max_iou:
                    max_iou = iou
                    best_pred_box = pred_box
            
            y_true.append(1)  # There is a ground truth box
            y_pred.append(1 if max_iou >= 0.5 else 0)  # Consider it a match if IoU >= 0.5

        for pred_box in predicted_boxes:
            iou_scores = [calculate_iou(pred_box, gt_box) for gt_box in ground_truth_boxes]
            if all(iou < 0.5 for iou in iou_scores):
                y_true.append(0)  # There was no matching ground truth box
                y_pred.append(1)  # False positive

    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }
    return metrics

# Evaluate the model
for iteration in iterations:
    print(f"{iteration.name}, id: {iteration.id}, Created at {iteration.created}, Last modified at {iteration.last_modified}")
    try:
        metrics = evaluate_model(test_data, predictor, project_id, iteration.id)
        print(metrics)
    except CustomVisionErrorException as e:
        print(f"Custom Vision Error: {e}")
        # Handle specific error scenario (e.g., invalid iteration ID)
    except Exception as ex:
        print(f"An unexpected error occurred: {ex}")
        # Handle other exceptions

    metrics = evaluate_model(test_data, predictor, project.id, iteration.id)
    print(metrics)
